In [27]:
import re
import operator
import itertools
from functools import reduce, partial

import numpy as np
from scipy import stats
import scipy
import pandas as pd
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
from bs4 import BeautifulSoup

# Config
pio.templates.default = "plotly_dark"

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("plotting.backend", "plotly")

# TODO: Backtesting

# Data

In [28]:
data = pd.read_csv("../data.csv")
data_male = data.groupby("College branch").get_group("Males")
data_female = data.groupby("College branch").get_group("Females")

data_filtered = data.drop(columns=data.columns[3:5])

courses_raw = ["ENGLISH100", "ARB100", "MATH101", "CHEM101", "STAT101", "TECH101", "ENTREPRENEUR101", "FAJAB101", "NAHAJ101", "ENGLISH110",
            "SALAM107", "PHYS104", "MATH106", "CSC111", "MATH151", "SALAM108", "CSC113", "CSC220", "MATH244",
            "CSC212", "CSC215", "MATH281", "CSC304", "CSC380", "CSC227", "CSC311", "CSC339", "CSC343", "CSC361",
            "CSC329", "CSC340", "CSC453", "CSC496", "PHYS210", "PHYS103", "CHEM103", "CSC443", "CSC462", "CSC476", "CSC478", "CSC484", "CSC489"]
criteria = ["Application", "Relevance", "Insight", "Understanding", "Ease"]
ratings = list(range(1, 6))

filtered_comments = ["I do not take this course. "]
filtered_courses = ["PHYS210", "CHEM103", "CSC484", "CSC476", "CSC478"]
bayes_weight = 2
name_to_figure = {}

ccis_df = pd.read_html("https://ccis.ksu.edu.sa/en/cs/Course-Catalog")

# Utility

In [29]:
@np.vectorize
def course_to_category(course):
    prefix_to_category = {"ENGLISH":"Humanities", "NAHAJ": "Humanities", "ARB": "Humanities", "ENTREPRENEUR": "Humanities", "FAJAB": "Humanities", "TECH": "Humanities",
                    "PHYS": "Physics", "CHEM": "Chemistry", "SALAM": "Islamics", "MATH": "Mathematics", "STAT": "Mathematics", "CSC": "Computer science"}
    return prefix_to_category[re.findall("[a-zA-Z]+", course)[0]]

def df_by_regex(df, regex):
    return df.filter(regex=regex, axis=1)

def get_subject_df(df, subject):
    return df_by_regex(df, f"{subject}.*\[.*\].*")

def remove_double_naming_in_legend(fig):
    for trace in fig.select_traces():
        trace.name = trace.name.split(",")[0]

def get_course_to_num_ranking_series(df):
    # a bit of a primitive way to count every filled in ranking (some rankings are NaN). Maybe it could be improved?
    i = 0
    course_to_num_ranking = {}
    while i < len(df):
        course_to_num_ranking.update({df.iloc[i].name[0]:len(df.iloc[i].dropna())})
        i += 5
    return pd.Series(course_to_num_ranking)

@np.vectorize
def build_df(cc):
    name = cc.iloc[0][0].replace(" ", "").split("–")[0].split("-")[0]
    lv1_indices = cc[0].copy()
    lv1_indices[0] = "Title"
    return pd.DataFrame(cc[1].values, index=pd.MultiIndex.from_tuples(zip(itertools.repeat(name), lv1_indices)))

def get_scores_df(df, criteria, categories):
    return pd.DataFrame(((bayesian_mean(df.loc[[(subject, criterion) for subject in courses]].T, categories)) for criterion in criteria), index=criteria)

def bayesian_mean(course_to_criterion, categories):
    course_to_criterion_mean = course_to_criterion.mean().droplevel(1)

    temp = course_to_criterion_mean.reset_index()
    temp["index"] = categories
    category_to_criterion_mean = temp.groupby("index").mean()[0]

    return course_to_criterion.apply(lambda x: scores_to_bayesian_mean(x, category_to_criterion_mean)).droplevel(1)

def scores_to_bayesian_mean(scores, category_to_criterion_mean, category_weight_portion=bayes_weight):
    """
    scores: Series of scores with Name formatted like this (Course, _)
    category_to_criterion_mean: category_to_criterion_mean
    category_weight_portion: the portion of scores category average is weighted at. e.g. 1 means its just as weighty as scores, 2 means half as weighty
    returns bayesian estimation of mean of scores, given its category
    """
    scores = scores.dropna()
    category_mean_series = pd.Series({"": category_to_criterion_mean[course_to_category(scores.name[0]).item(0)]})
    weights = [max(len(scores) // category_weight_portion, 1)] + [1] * len(scores)
    normalized_weights = weights/np.linalg.norm(weights)
    return np.average(scores.append(category_mean_series), weights=normalized_weights)


# Wrangling

In [30]:
# Dataframes
index = list((course, criterion) for criterion in criteria for course in courses_raw)
sorted_index = sorted(index, key=lambda x: courses_raw.index(x[0]))
df = get_subject_df(data_filtered, "").T.set_index(pd.Index(sorted_index)).drop(filtered_courses)

# Useful structures
courses = courses_raw.copy()
[courses.remove(course) for course in filtered_courses]
categories = pd.Series(course_to_category(courses))
assert sum(categories.isna()) == 0, "prefix_to_category must cover each course prefix"

# Comments
course_to_comments = {}
other_details_df = df_by_regex(data_filtered, "Other details you want to add.*")
for course, other_details_col in zip(courses, other_details_df):
    filt = (other_details_df[other_details_col].isna()) | reduce(operator.or_, (comment == other_details_df[other_details_col] for comment in filtered_comments))
    comments = other_details_df[other_details_col][~filt]
    if comments.all():
        course_to_comments[course] = comments

# Course info
course_info = pd.concat(build_df(ccis_df[:-14]))
course_info.columns = ["Values"]
course_info.loc[(("ENGLISH100", "Title"), "Values")] = "English 100"
course_info.loc[(("ARB100", "Title"), "Values")] = "Arabic 100"
course_info.loc[(("MATH101", "Title"), "Values")] = "Math 101 - Derivative Math"
course_info.loc[(("CHEM101", "Title"), "Values")] = "Chemistry 101 - Intro To Chemistry"
course_info.loc[(("STAT101", "Title"), "Values")] = "Statistics 101"
course_info.loc[(("TECH101", "Title"), "Values")] = "Tech 140 - Computer Skills"
course_info.loc[(("ENTREPRENEUR101", "Title"), "Values")] = "Entrepreneurship 101"
course_info.loc[(("FAJAB101", "Title"), "Values")] = "Fajab 101 - Health and Stuff"
course_info.loc[(("NAHAJ101", "Title"), "Values")] = "Nahaj 101 - University Skills"
course_info.loc[(("ENGLISH110", "Title"), "Values")] = "English 110"
course_info.loc[(("SALAM107", "Title"), "Values")] = "Salam 107 - Job Ethics"
course_info.loc[(("PHYS104", "Title"), "Values")] = "Physics 104 - Electrical Physics"
course_info.loc[(("MATH106", "Title"), "Values")] = "Math 106 - Integral Math"
course_info.loc[(("MATH151", "Title"), "Values")] = "Math 151 - Discrete Math"
course_info.loc[(("SALAM108", "Title"), "Values")] = "Salam 108 - Modern Issues"
course_info.loc[(("MATH244", "Title"), "Values")] = "MATH 244 - Linear Algebra"
course_info.loc[(("MATH281", "Title"), "Values")] = "Math 281 - Combinatorial Math"
course_info.loc[(("CSC343", "Title"), "Values")] = "CSC 343 - Intro To Software Engineering"
course_info.loc[(("PHYS103", "Title"), "Values")] = "Physics 103 - Newtonian Physics"
course_info.loc[(("CSC489", "Title"), "Values")] = "CSC 489 - Selected Topics in Computer Science"
course_to_info = {}
for course in courses:
    course_to_info[course] = course_info.loc[course].values[0][0]
course_info_df = pd.DataFrame(course_to_info.values(), course_to_info.keys(), columns=["Description"])

D:\Softwarez\Anaconda\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.

D:\Softwarez\Anaconda\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



## Ranking

### Course ranking by criteria

In [31]:
functions = [
            lambda x: bayesian_mean(x, categories), 
            lambda x: pd.Series(np.nanmedian(x, axis=0), index=courses), 
            lambda x: pd.Series(stats.mode(x, nan_policy="omit")[0].data[0], index=courses),
            lambda x: x.std().droplevel(1),
]
function_to_name = {function: function_name for function, function_name in zip(functions, ["Bayesian estimated mean", "Median", "Mode", "Standard deviation"])}
rows = len(functions) * 3
cols = 2
subtitles = np.empty((rows, cols), dtype="object")
for f_i, function in enumerate(functions):
    for c_i, criterion in enumerate(criteria):
        row = f_i * 3 + int(c_i / 2)
        col = c_i % 2
        subtitles[row, col] = (f"{function_to_name.get(function)} {criterion} score for each course")


fig_ranking = make_subplots(rows=len(functions) * 3, cols=2, 
        subplot_titles=subtitles.reshape(-1))
        
for f_i, function in enumerate(functions):
    for c_i, criterion in enumerate(criteria):
        row = f_i * 3 + int(c_i / 2 + 1)
        col = c_i % 2 + 1
        course_to_criterion = function(df.loc[[(subject, criterion) for subject in courses]].T)

        scatter = px.scatter(course_to_criterion, color=categories, symbol=categories, hover_name=course_info_df.Description)
        for trace in scatter.select_traces():
            # Only show legend once
            if f_i == c_i == 0:
                trace.showlegend = True
            else:
                trace.showlegend = False

            fig_ranking.add_trace(
                trace,
                row=row, col=col
            )
        fig_ranking.update_yaxes(
            title=f"{function_to_name.get(function)} {criterion}",
            row=row, col=col
        )
    remove_double_naming_in_legend(fig_ranking)
    fig_ranking.update_layout(height=5000, width=1920)

name = "ranking_of_criteria"
fig_ranking.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_ranking})
fig_ranking

### Scores 

In [32]:
scores_df = get_scores_df(df, criteria, categories)

In [33]:
i = 0
course_to_num_ranking = {}
while i < len(df):
    course_to_num_ranking.update({df.iloc[i].name[0]:len(df.iloc[i].dropna())})
    i += 5
course_to_num_ranking_series = pd.Series(course_to_num_ranking)

In [34]:
general_scores = scores_df.sum().sort_values() / 25
sorted_categories = course_to_category(general_scores.index)
fig_scores = px.scatter(general_scores, color=sorted_categories, symbol=sorted_categories, title="Course ranking (General)", hover_name=course_info_df.Description)
fig_scores.update_xaxes(title="Course")
fig_scores.update_yaxes(title="Ranking")

remove_double_naming_in_legend(fig_scores)

fig_scores.add_trace(
    go.Bar(x=course_to_num_ranking_series.index, y=course_to_num_ranking_series / np.linalg.norm(course_to_num_ranking_series), marker={"color": "rgba(100, 100, 100, 0.3)"}, name="Percentage of data points"),
    # row=2, col=1
)
fig_scores.update_layout(height=980, width=1920)

name = "Course ranking (General)"
fig_scores.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_scores})
fig_scores

### Weighted scores

In [35]:
weights_real_world = {"Application": 1.5, "Relevance": 1, "Understanding": 0.2, "Ease": 0.5}
weights_academic = {"Insight": 1.5, "Understanding": 1.5}

scores_df_real_world = scores_df.apply(lambda x: weights_real_world.get(x.name, 0) * x, axis=1)
scores_df_academic = scores_df.apply(lambda x: weights_academic.get(x.name, 0) * x, axis=1)

standardized_scores_df_real_world = scores_df_real_world.sum() / max(scores_df_real_world.sum())
standardized_scores_df_academic = scores_df_academic.sum() / max(scores_df_academic.sum())

In [36]:
weighted_scores = pd.DataFrame([standardized_scores_df_real_world, standardized_scores_df_academic]).T
weighted_scores.columns = ("Real-world score", "Academic score")

In [37]:
sorted_categories = course_to_category(weighted_scores.index)
fig_weighted_scores = px.scatter(weighted_scores, x="Real-world score", y="Academic score", color=sorted_categories, symbol=sorted_categories, title="Course ranking (Real-world Vs. Academic)",
                hover_name=course_info_df.Description, text=weighted_scores.index)

remove_double_naming_in_legend(fig_weighted_scores)

fig_weighted_scores.update_traces(textposition="top center")
fig_weighted_scores.update_layout(height=980, width=1920)
fig_weighted_scores.add_annotation(
    x=.4, y=.98,
    text="Academic_score = (Insight: 1.5 Understanding * 1.5) / max_score",
    showarrow=False
)
fig_weighted_scores.add_annotation(
    x=.462, y=1,
    text="Real_world_score = (Application * 1.5 + Relevance + Understanding * 0.2 + Ease * 0.5) / max_score",
    showarrow=False
)

name = "Course ranking (Real-world Vs. Academic)"
fig_weighted_scores.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_weighted_scores})
fig_weighted_scores

# Comments

In [38]:
fig_comments = go.Figure()
count = 1
for course, comments in course_to_comments.items():
    if len(comments):
        fig_comments.add_annotation(
            x=1, y=count,
            text=course,
            showarrow=False,
        )
        for comment in comments:
            fig_comments.add_annotation(
                x=7, y=count,
                text=comment,
                showarrow=False,
            )
            count += 1
fig_comments.update_xaxes(showgrid=False)
fig_comments.update_yaxes(showgrid=False)
fig_comments.update_layout(height=980, width=1920, yaxis_range=[0, count], xaxis_range=[0, 10], title="Comments on courses")

name = "comments"
fig_comments.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_comments})
fig_comments

In [39]:
scores_df_matrix = scores_df.T
scores_df_matrix["categories"] = categories.values
fig_matrix = px.scatter_matrix(scores_df_matrix, dimensions=criteria, symbol="categories", color="categories", hover_name=course_info_df.Description, title="Scatter matrix of all criteria")
fig_matrix.update_layout(height=980, width=1920)

name = "scatter_matrix_criteria"
fig_matrix.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_matrix})
fig_matrix

In [40]:
fig_categories_criteria_means = px.imshow(scores_df_matrix.groupby("categories").mean(), title="Categories summary")
fig_categories_criteria_means.update_layout(height=980, width=1920)
name = "Categories summary"
fig_categories_criteria_means.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_categories_criteria_means})
fig_categories_criteria_means

In [41]:
all_scores = weighted_scores.copy()
all_scores["General score"] = general_scores
all_scores["categories"] = categories.values
fig_scatter_3d = px.scatter_3d(all_scores, x="General score", y="Real-world score", z="Academic score", symbol="categories", color="categories", hover_name=course_info_df.Description, title="3D-Scatter plot of all scores")
fig_scatter_3d.update_layout(height=980, width=1920)
name = "3d_scatter_scores"
fig_scatter_3d.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_scatter_3d})
fig_scatter_3d

In [42]:
fig_corr = px.imshow(all_scores.corr(), title="Correlation of score types")
fig_corr.update_layout(height=980, width=1920)
name = "Correlation of score types"
fig_corr.write_image(f"images/{name}.png")
name_to_figure.update({name: fig_corr})
fig_corr

# Generate site

In [43]:
with open("index_empty.html.test") as file:
    html = file.read()

In [44]:
with open("index.html", "w") as file:
    file.write(html)
    for figure in name_to_figure.values():
        file.write(figure.to_html(full_html=False, include_plotlyjs=False))
    file.write("</div></body></html>")

# Meta stats

In [45]:
get_course_to_num_ranking_series(df).describe()

count    37.000000
mean     10.000000
std       3.109126
min       2.000000
25%       8.000000
50%      11.000000
75%      13.000000
max      14.000000
dtype: float64

# Research stuff

In [46]:
print(all_scores.to_latex())

\begin{tabular}{lrrrl}
\toprule
{} &  Real-world score &  Academic score &  General score &        categories \\
\midrule
ENGLISH100      &          0.977969 &        0.587055 &       0.749899 &        Humanities \\
ARB100          &          0.644273 &        0.561107 &       0.559423 &        Humanities \\
MATH101         &          0.899391 &        0.883593 &       0.772007 &       Mathematics \\
CHEM101         &          0.520728 &        0.438133 &       0.445344 &         Chemistry \\
STAT101         &          0.927789 &        0.804526 &       0.756429 &       Mathematics \\
TECH101         &          0.765262 &        0.545605 &       0.626730 &        Humanities \\
ENTREPRENEUR101 &          0.601895 &        0.521720 &       0.530546 &        Humanities \\
FAJAB101        &          0.600165 &        0.538924 &       0.546742 &        Humanities \\
NAHAJ101        &          0.638404 &        0.661528 &       0.594242 &        Humanities \\
ENGLISH110      &          0.918

In [47]:
print(all_scores.to_markdown())

|                 |   Real-world score |   Academic score |   General score | categories       |
|:----------------|-------------------:|-----------------:|----------------:|:-----------------|
| ENGLISH100      |           0.977969 |         0.587055 |        0.749899 | Humanities       |
| ARB100          |           0.644273 |         0.561107 |        0.559423 | Humanities       |
| MATH101         |           0.899391 |         0.883593 |        0.772007 | Mathematics      |
| CHEM101         |           0.520728 |         0.438133 |        0.445344 | Chemistry        |
| STAT101         |           0.927789 |         0.804526 |        0.756429 | Mathematics      |
| TECH101         |           0.765262 |         0.545605 |        0.62673  | Humanities       |
| ENTREPRENEUR101 |           0.601895 |         0.52172  |        0.530546 | Humanities       |
| FAJAB101        |           0.600165 |         0.538924 |        0.546742 | Humanities       |
| NAHAJ101        |           